# Equal Weighted Portfolio
Each of N stocks in the portfolio receives a 1/N allocation 


## Imports

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from matplotlib import pyplot as plt
plt.style.use('seaborn-v0_8')

## Data

In [ ]:
df_stocks = yf.download(['META', 'AMZN', 'AAPL', 'NFLX', 'GOOG'], start='2015-01-01', end='2025-12-31')['Close']
df_stocks.head()

In [ ]:
df_stocks.plot(figsize=(12,8))

## Creating the EWP

In [ ]:
# Calculating the daily returns of each stock

df_returns = df_stocks.pct_change().dropna()
df_returns.head()

In [ ]:
# Each asset is given the same monetary investment equal to 1/N of the total investment amount

num_assets = len(df_stocks.columns)
weights = [1/num_assets for i in range(num_assets)]

In [ ]:
# We can calculate the equal weighted portfolio returns by refactoring the returns by the equal weight and summing them together i.e. the dot product

df_returns['EWP'] = df_returns.dot(weights)
df_returns.head()

In [ ]:
summary = df_returns.agg(['mean', 'std']).T
summary.columns = ['Return', 'Risk']
summary

In [ ]:
ann_summary = summary.copy()
ann_summary['Return'] = ann_summary['Return'] * 252
ann_summary['Risk'] = ann_summary['Risk'] * np.sqrt(252)
ann_summary['Sharpe'] = (ann_summary['Return'] - 0) / ann_summary['Risk']
ann_summary

In [ ]:
ann_summary.plot(kind = "scatter", x = "Risk", y = "Return", figsize = (13,9), s = 50, fontsize = 15)
for i in ann_summary.index:
    plt.annotate(i, xy=(ann_summary.loc[i, "Risk"]+0.002, ann_summary.loc[i, "Return"]+0.002), size = 15)
plt.xlabel("ann. Risk", fontsize = 15)
plt.ylabel("ann. Return", fontsize = 15)
plt.title("Risk/Return", fontsize = 20)
plt.show()